In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import re
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from oauth2client import file, client, tools
import os
import httplib2
import glob


In [15]:
# Run a new search, and extract the corresponding search id
# search_string = 'https://search.larryhotz.com/search/results/?city=Denver&zip=all&neighborhood=all&type=res&type=con&search_status=SOLD&sold_price_min=all&sold_price_max=all&sold_within=1000&short_sale=all&beds_min=all&baths_min=all&area_min=all&amenities=all&lot_size_min=all&year_built_min=all&levels=all&school_district=all&basement=all&sort_latest=true'

'/4bc6/'

In [65]:

total_search = 'https://search.larryhotz.com/search/results/?city=all&zip=all&neighborhood=all&type=res&type=con&type=mul&search_status=SOLD&sold_price_min=all&sold_price_max=all&sold_within=all&short_sale=all&beds_min=all&baths_min=all&area_min=all&amenities=all&lot_size_min=all&year_built_min=all&levels=all&school_district=all&basement=all'
# zip_code_search

def retrieve_search_properties(search_string):
    search_page = requests.get(search_string)
    search_html1 = soup(search_page.content, 'html.parser')
    search_html = search_html1.find('a', class_="savesearch_button")['href']
    search_code = re.findall("/\w{4}/", search_html)[0]
    nprop = soup(search_page.content,'html.parser').find('p',class_='results_properties_found').text
    charn = re.findall(r'\d+',nprop)
    n = int("".join(charn))
    if n > 9980:
        print('Warning - too many results for this search to parse all')
    return(search_code, n)

def clean_dat_string(value):
    value = value.text
    result = re.sub(r'[\t\r\n]', '', value).strip()
    return result

def parse_prop(prop_details, label):
    value = prop_details.find(text=label)
    if value is not None: 
        value = value.next_element.next_element.text
    return value

In [118]:
# Manual scrape function
# Loops through urls, scrapes the data, and saves it to the dataframe

def scrape_properties(zip_code,search_code,max_results):
    out_list=[]
    for i in range(0,max_results):
        result_url = 'https://search.larryhotz.com/search/details'+search_code+str(i)
        result_page = requests.get(result_url)
        result_page = soup(result_page.content, 'html.parser')
        try:
            prop_details = result_page.findAll(class_='property_detail_specs')[0]
            prop_details2 = result_page.findAll(class_='property_detail_specs')[1]
            sa1 = clean_dat_string(result_page.find('h2'))
            sa2 = clean_dat_string(result_page.find('h3'))
        except Exception:
            pass
        outdict = {
            'zip_code': str(zip_code), 
            'sa1' :sa1,
            'sa2' :sa2,
            'closing_date':parse_prop(prop_details,'Closing Date'),
            'list_price': parse_prop(prop_details,'List Price'),
            'sold_price': parse_prop(prop_details,'Sold Price'),
            'prop_type': parse_prop(prop_details,'Type'),
            'year_built': parse_prop(prop_details,'Year Built'),
            'sq_ft': parse_prop(prop_details,'Total Sqft'),
            'bedrooms': parse_prop(prop_details,'Bedrooms'),
            'bathrooms': parse_prop(prop_details,'Bathrooms'),
            'sub_div': parse_prop(prop_details,'Subdivision'),
            'sub_area': parse_prop(prop_details,'Sub-Area'),
            'year_built': parse_prop(prop_details,'Year Built'),
            'taxes': parse_prop(prop_details2,'Taxes'),
            'school_district': parse_prop(prop_details2,'School District'),
            'garage_spaces': parse_prop(prop_details2,'Garage Spaces'),
            'parking_spaces': parse_prop(prop_details2,'Parking Spaces'),
            'basement':parse_prop(prop_details2,'Basement'),
            'hoa_fees': parse_prop(prop_details2,'Total HOA Fees'),
            'financial_terms': parse_prop(prop_details2,'Financial Terms')
            }
        out_list.append(outdict)
        if (i % 10 ==0):
            print ("{} results recorded for {} postal code".format(i,zip_code))
    print("{} fully scraped. {} transactions recorded".format(zip_code,i))
    result_df = pd.DataFrame(out_list)
    return(out_list,result_df)

In [135]:
all_zips = ['08230','09828','18011','80000','80002','80003','80004','80005','80007','80010','80011','80012','80013','80014','80015','80016','80017','80018','80019','80020','80021','80022','80023','80024','80025','80026','80027','80028','80029','80030','80031','80033','80049','80101','80102','80103','80104','80105','80106','80107','80108','80109','80110','80111','80112','80113','80114','80116','80117','80118','80120','80121','80122','80123','80124','80125','80126','80127','80128','80129','80130','80131','80132','80133','80134','80135','80136','80137','80138','80149','80156','80166','80189','80202','80203','80204','80205','80206','80207','80209','80210','80211','80212','80214','80215','80216','80218','80219','80220','80221','80222','80223','80224','80226','80227','80228','80229','80230','80231','80232','80233','80234','80235','80236','80237','80238','80239','80241','80244','80246','80247','80249','80250','80260','80301','80302','80303','80304','80305','80311','80333','80401','80402','80403','80405','80407','80419','80420','80421','80422','80424','80425','80427','80428','80430','80432','80433','80434','80435','80436','80437','80438','80439','80440','80442','80443','80444','80445','80446','80447','80448','80449','80451','80452','80453','80454','80455','80456','80457','80459','80461','80465','80466','80467','80468','80469','80470','80471','80473','80474','80475','80476','80477','80478','80479','80480','80481','80482','80483','80487','80497','80498','80499','80500','80501','80502','80503','80504','80510','80512','80513','80514','80515','80516','80517','80520','80521','80524','80525','80526','80528','80530','80531','80532','80534','80535','80536','80537','80538','80540','80541','80542','80543','80544','80545','80546','80547','80549','80550','80587','80601','80602','80603','80604','80610','80611','80612','80615','80620','80621','80622','80623','80624','80631','80634','80640','80642','80643','80644','80645','80646','80648','80649','80650','80651','80652','80653','80654','80701','80705','80720','80722','80723','80726','80727','80728','80729','80731','80732','80733','80734','80735','80736','80737','80741','80742','80743','80744','80745','80746','80747','80749','80750','80751','80754','80755','80757','80758','80759','80787','80801','80804','80805','80807','80808','80809','80810','80812','80813','80814','80815','80816','80817','80818','80819','80820','80821','80822','80823','80824','80825','80827','80828','80829','80830','80831','80832','80833','80834','80835','80836','80842','80860','80861','80863','80864','80866','80901','80903','80904','80905','80906','80907','80908','80909','80910','80911','80915','80916','80917','80918','80919','80920','80921','80922','80923','80924','80925','80926','80927','80928','80929','80930','80938','80939','80951','81001','81003','81004','81005','81006','81007','81008','81019','81020','81022','81023','81024','81025','81030','81036','81039','81040','81041','81045','81046','81050','81052','81054','81055','81058','81059','81062','81063','81064','81067','81069','81071','81073','81076','81077','81081','81082','81089','81091','81101','81105','81120','81122','81123','81124','81125','81130','81131','81132','81133','81134','81136','81137','81138','81140','81141','81143','81144','81146','81147','81148','81149','81151','81152','81153','81154','81155','81201','81211','81212','81215','81220','81221','81222','81223','81224','81225','81226','81227','81228','81230','81232','81233','81235','81236','81240','81242','81243','81244','81249','81251','81252','81253','81255','81301','81321','81323','81328','81401','81403','81410','81411','81413','81415','81416','81418','81419','81423','81424','81425','81428','81430','81431','81432','81435','81501','81503','81504','81505','81506','81507','81520','81521','81522','81523','81524','81525','81526','81527','81601','81610','81611','81615','81620','81621','81623','81624','81625','81626','81630','81631','81632','81635','81637','81638','81639','81640','81641','81643','81645','81646','81647','81648','81649','81650','81652','81653','81654','81657','82063','82084','82325','84027','84066','85121','89074','89110','90109','91089']
short_zips = all_zips[0:5]
short_zips_df = pd.DataFrame(short_zips)
zip_list = []

for x in all_zips[15:30]:
    search_url = 'https://search.larryhotz.com/search/results/?city=all&zip={}&neighborhood=all&type=res&type=con&type=mul&search_status=SOLD&sold_price_min=all&sold_price_max=all&sold_within=all&short_sale=all&beds_min=all&baths_min=all&area_min=all&amenities=all&lot_size_min=all&year_built_min=all&levels=all&school_district=all&basement=all'.format(x)
    search_code,max_results = retrieve_search_properties(search_url)
    if max_results == 0:
        print('no results available for {}'.format(x))
        results = 'none'
    elif max_results > 9980 :
        print('Too many results available for {}'.format(x))
        results = 'too many'
    else:
        print('{} results available for {}'.format(max_results,x))
        results = 'within range'
        out_list=[]
        
        for i in range(0,max_results):
            result_url = 'https://search.larryhotz.com/search/details'+search_code+str(i)
            result_page = requests.get(result_url)
            result_page = soup(result_page.content, 'html.parser')
            try:
                prop_details = result_page.findAll(class_='property_detail_specs')[0]
                prop_details2 = result_page.findAll(class_='property_detail_specs')[1]
                sa1 = clean_dat_string(result_page.find('h2'))
                sa2 = clean_dat_string(result_page.find('h3'))
            except Exception:
                pass
            outdict = {
                'zip_code': str(x), 
                'sa1' :sa1,
                'sa2' :sa2,
                'closing_date':parse_prop(prop_details,'Closing Date'),
                'list_price': parse_prop(prop_details,'List Price'),
                'sold_price': parse_prop(prop_details,'Sold Price'),
                'prop_type': parse_prop(prop_details,'Type'),
                'year_built': parse_prop(prop_details,'Year Built'),
                'sq_ft': parse_prop(prop_details,'Total Sqft'),
                'bedrooms': parse_prop(prop_details,'Bedrooms'),
                'bathrooms': parse_prop(prop_details,'Bathrooms'),
                'sub_div': parse_prop(prop_details,'Subdivision'),
                'sub_area': parse_prop(prop_details,'Sub-Area'),
                'year_built': parse_prop(prop_details,'Year Built'),
                'taxes': parse_prop(prop_details2,'Taxes'),
                'school_district': parse_prop(prop_details2,'School District'),
                'garage_spaces': parse_prop(prop_details2,'Garage Spaces'),
                'parking_spaces': parse_prop(prop_details2,'Parking Spaces'),
                'basement':parse_prop(prop_details2,'Basement'),
                'hoa_fees': parse_prop(prop_details2,'Total HOA Fees'),
                'financial_terms': parse_prop(prop_details2,'Financial Terms')
                }
            out_list.append(outdict)
            if (i % 10 ==0):
                print ("{} results recorded for {} postal code".format(i,x))
        scrapes_df = pd.DataFrame(out_list)
        print("{} fully scraped. {} transactions recorded".format(x,i))
        file_path = '/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results/results-{}-1.csv'.format(x)
        scrapes_df.to_csv(file_path)
    zip_dict = {
        'zip_code':x,
        'url_code':search_code,
        'url':search_url,
        'max_results': max_results,
        'outcome': results,
        'scrapes_list': scrapes_list,
        }
    zip_list.append(zip_dict)
    zip_list_df = pd.DataFrame(zip_list)

4398 results available for 80016
0 results recorded for 80016 postal code
10 results recorded for 80016 postal code
20 results recorded for 80016 postal code
30 results recorded for 80016 postal code
40 results recorded for 80016 postal code
50 results recorded for 80016 postal code
60 results recorded for 80016 postal code
70 results recorded for 80016 postal code
80 results recorded for 80016 postal code
90 results recorded for 80016 postal code
100 results recorded for 80016 postal code
110 results recorded for 80016 postal code
120 results recorded for 80016 postal code
130 results recorded for 80016 postal code
140 results recorded for 80016 postal code
150 results recorded for 80016 postal code
160 results recorded for 80016 postal code
170 results recorded for 80016 postal code
180 results recorded for 80016 postal code
190 results recorded for 80016 postal code
200 results recorded for 80016 postal code
210 results recorded for 80016 postal code
220 results recorded for 80016 p

1880 results recorded for 80016 postal code
1890 results recorded for 80016 postal code
1900 results recorded for 80016 postal code
1910 results recorded for 80016 postal code
1920 results recorded for 80016 postal code
1930 results recorded for 80016 postal code
1940 results recorded for 80016 postal code
1950 results recorded for 80016 postal code
1960 results recorded for 80016 postal code
1970 results recorded for 80016 postal code
1980 results recorded for 80016 postal code
1990 results recorded for 80016 postal code
2000 results recorded for 80016 postal code
2010 results recorded for 80016 postal code
2020 results recorded for 80016 postal code
2030 results recorded for 80016 postal code
2040 results recorded for 80016 postal code
2050 results recorded for 80016 postal code
2060 results recorded for 80016 postal code
2070 results recorded for 80016 postal code
2080 results recorded for 80016 postal code
2090 results recorded for 80016 postal code
2100 results recorded for 80016 

3750 results recorded for 80016 postal code
3760 results recorded for 80016 postal code
3770 results recorded for 80016 postal code
3780 results recorded for 80016 postal code
3790 results recorded for 80016 postal code
3800 results recorded for 80016 postal code
3810 results recorded for 80016 postal code
3820 results recorded for 80016 postal code
3830 results recorded for 80016 postal code
3840 results recorded for 80016 postal code
3850 results recorded for 80016 postal code
3860 results recorded for 80016 postal code
3870 results recorded for 80016 postal code
3880 results recorded for 80016 postal code
3890 results recorded for 80016 postal code
3900 results recorded for 80016 postal code
3910 results recorded for 80016 postal code
3920 results recorded for 80016 postal code
3930 results recorded for 80016 postal code
3940 results recorded for 80016 postal code
3950 results recorded for 80016 postal code
3960 results recorded for 80016 postal code
3970 results recorded for 80016 

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [141]:
# 80004 scraped up to 960. Buuut I guess that doesn't mean anything.
# 80015 scraped up to 4506 of 4898. 

# all_zips[6:9]
scrapes_df_manual = pd.DataFrame(out_list)
file_path_manual = '/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results/results-{}-1.csv'.format('80016')
scrapes_df_manual.to_csv(file_path_manual)

In [117]:


# Combines the recorded csvs into a single dataframe
csv_list = glob.glob('/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results/*.csv')
base_df = pd.DataFrame([])
for doc in csv_list:
    new_csv = pd.read_csv(doc)
    base_df = pd.concat([base_df,new_csv])
    print(doc)
    
base_df

/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results/results-80101-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results/results-08230-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results/results-18011-1.csv


,Unnamed: 0,zip_code,sa1,sa2,closing_date,list_price,sold_price,prop_type,year_built,sq_ft,...,bathrooms,sub_div,sub_area,taxes,school_district,garage_spaces,parking_spaces,basement,hoa_fees,financial_terms
0,0,80101.0,23150 E Lost Creek Trail,"Agate, CO 80101","Mar 05, 2021","$599,950","$625,000",Single Family Residential,2003,2581,...,3.0,Ponderosa Ranges,Ponderosa Ranges,"$2,293",Kiowa C-2,2.0,2.0,"Exterior Entry, Finished, Partial, Walk-Out Ac...",NaN,Conventional
1,1,80101.0,36101 County Road 162,"Agate, CO 80101","Sep 30, 2020","$650,000","$617,000",Single Family Residential,2002,4330,...,3.0,Rurala,Rurala,"$2,181",Agate 300,3.0,3.0,"Exterior Entry, Full, Unfinished, Walk-Out Access",NaN,VA
2,2,80101.0,38825 Rusty Spur Trail,"Agate, CO 80101","Apr 27, 2020","$525,000","$526,000",Single Family Residential,2004,2668,...,2.0,Ponderosa Ranges,Ponderosa Ranges,"$1,498",Kiowa C-2,3.0,3.0,"Daylight, Full, Finished, Walk-Out Access",NaN,Conventional
3,3,80101.0,39450 County Road 162,"Agate, CO 80101","Mar 19, 2021","$499,000","$500,000",Single Family Residential,2007,1805,...,2.0,Agate,Agate,$773,Agate 300,NaN,NaN,Crawl Space,NaN,Conventional
4,4,80101.0,27990 Private Road 139,"Agate, CO 80101","Oct 20, 2020","$499,000","$483,000",Single Family Residential,2003,4914,...,5.0,Sunset Ranch,Sunset Ranch,"$1,890",Agate 300,2.0,2.0,"Exterior Entry, Unfinished, Walk-Out Access",NaN,FHA
5,5,80101.0,41501 County Road 153,"Agate, CO 80101","Apr 26, 2021","$399,900","$425,000",Single Family Residential,1999,3024,...,2.0,Rural,Rural,$880,Agate 300,6.0,6.0,"Daylight, Exterior Entry, Full, Bath/Stubbed, ...",NaN,Conventional
6,6,80101.0,38441 County Road 166,"Agate, CO 80101","Mar 26, 2020","$299,900","$299,900",Single Family Residential,2003,2692,...,3.0,Healds Add Amd No2,Healds Add Amd No2,$650,Agate 300,2.0,2.0,"Full, Finished",NaN,Other
7,7,80101.0,36995 County Road 162,"Agate, CO 80101","Dec 06, 2018","$225,000","$205,000",Single Family Residential,1942,5836,...,2.0,Agate,Agate,$881,Agate 300,NaN,NaN,"Finished, Partial",NaN,Conventional
8,8,80101.0,40786 1st Avenue,"Agate, CO 80101","Feb 24, 2021","$160,000","$152,000",Single Family Residential,1922,972,...,2.0,Agate Original,Agate Original,$271,Agate 300,NaN,NaN,NaN,NaN,USDA
9,9,80101.0,38595 Monroe Street,"Agate, CO 80101","Nov 30, 2020","$128,500","$130,500",Single Family Residential,1953,1856,...,1.0,Agate Petersons Add,Agate Petersons Add,$475,Agate 300,NaN,NaN,Unfinished,NaN,FHA


'/4bck/'

In [79]:
search_html

'/member/save_search/4bc6/'

In [35]:
for x in all_zips:
    file_path = '/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-{}-1.csv'.format(x)
    print (file_path)


/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-08230-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80000-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80010-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80012-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80022-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80026-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80110-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80111-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80113-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80123-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/denver_scrape/results-80127-1.csv
/Users/dmitri.mirakyanopendoor.com/Documents/GitHub/de

In [ ]:
# Initiate a list of all possible zip codes
# For each search - generate a zip code
# Then identify the # properties sold within that zip code
# Find the search url code for that website
# Generate dataframe 

In [49]:
scopes = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_key_filename = 'denver-311215-14d0c79b4aa2.json'

credentials = Credentials.from_service_account_file(json_key_filename, scopes=scopes)
service = build('sheets', 'v4', credentials=credentials)
gc = gspread.authorize(credentials)


In [51]:
client_email = 'create-sales@denver-311215.iam.gserviceaccount.com'
spreadsheet_key = '1jRBmZigXTJHO5Y7HoJ6kRnyTSekZ4BS7je1lwfKqpRg'
wks_name = 'Sheet1'

sh1 = gc.open_by_key(spreadsheet_key).get_worksheet(0)
df_test = dataframe = pd.DataFrame(sh1.get_all_records())
df_test

set_with_dataframe(sh1, result_df)